In [6]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode

In [7]:
pd.set_option('display.max_rows', 500)

In [ ]:
df = pd.read_csv("/home/mila/PycharmProjects/Python-codes/Data_scraping/tl_ready.csv", sep=',', low_memory=False, encoding='utf-8', na_values=' ')
df = df.applymap(str)

for i in df.columns:
    df[i] = df[i].str.replace('=', '')
    df[i] = df[i].str.replace('"', '')
    df[i] = df[i].str.replace('«', 'left')
    df[i] = df[i].str.replace('»', 'right')
    df[i] = df[i].str.replace('None', '')
    df[i] = df[i].str.replace('nan', '')  

df = df.replace(np.nan, '')

del df['Unnamed: 0']
df1 = df['date']
del df['date']
df['all'] = df[df.columns].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
df = df['all'].to_frame()
df = pd.concat([df1,df], axis=1)
df.to_excel("10_days1.xlsx")
# len(df[df.num.notnull()]) 
#dictionary = {'Ã': 'a'}
#df = df.replace(dictionary, regex=False, inplace=True)
#df = df.style.set_properties(subset=['all'], **{'width': '300px'})

In [577]:
# extract observations with 4digits
df[['4num1', '4num2']] = df['all'].str.extract(r'(?<!\d)(\d{1})[^\S](\d{3})', expand=False)
df4num1 = df[df['4num1'].notnull()]
df4num1['4num'] = df4num1['4num1'] + df4num1['4num2']
df4num1

/home/mila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,date,all,4num1,4num2,4num
94,tl60.06.01.2.png,"3 v . mod . mobl . leil , tin . 20/621/8 kr . ...",1,000,1000


In [578]:
df['4num3'] = df['all'].str.extract(r'(?<!\d)(\d{4})(?!\d)', expand=False)
df4num2 = df[df['4num3'].notnull()]
df4num2['4num'] = df4num2['all'].str.extract(r'[/k/][/r/][^\S][/.][^\S](\d{4})(?!\d)', expand=False)
df4num2 = df4num2[df4num2['4num'].notnull()]
df4num2

/home/mila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,date,all,4num1,4num2,4num3,4num
31,tl60.06.10.1.png,"Stor , umøblert villa på Blommenholm 3 stuer ,...",NaN,NaN,1961,1000


In [583]:
#add a column in df with only 3 extracted digits - candidates for rents
df['3num'] = df['all'].str.extract(r'(?<!\d)(\d{3})(?!\d)', expand=False)
#form new df with them
df3num = df[df['3num'].notnull()]
# concat wih 4 num
df3num = pd.concat([df3num, df4num1, df4num2]).reset_index(drop=True)
df3num = df3num.drop(['4num1', '4num2', '4num3'], axis = 1) 
#extract all with slash space with space
df3num['slash_space'] = df3num['all'].str.extract(r'\/[^\S](\d{3})(?!\d)', expand=False)
#extract all with slash space
df3num['slash'] = df3num['all'].str.extract(r'\/(?<!\d)(\d{3})(?!\d)!\/', expand=False)
#extract 3 digits, space and zeros
df3num['zeros'] = df3num['all'].str.extract(r'[/0/][/0/][^\S](?<!\d)(\d{3})(?!\d)', expand=False)
#delete prices with 3 digits, space and 000
re_express = re.compile('([\d]{3})[^\S][/0/][/0/][/0/].*')
df3num['all'] = df3num['all'].str.replace(re_express, '')
#find all with kr . 3dig
df3num['kr'] = df3num['all'].str.extract(r'[/kr/][^\S][/.][^\S](\d{3})(?!\d)', expand=False)
#find all with kr after  (\b kr$)
df3num['kr_after'] = df3num['all'].str.extract(r'(?<!\d)(\d{3})[^\S][/k][/r]', expand=False)
#find all with kı . 3dig
df3num['ki'] = df3num['all'].str.extract(r'[/kı/][^\S][/.][^\S](\d{3})(?!\d)', expand=False)
#find leie 3dig - choose others too - why?
df3num['leie'] = df3num['all'].str.extract(r'[/leie/][^\S](\d{3})(?!\d)', expand=False)
#match rent to one column
df3num['rent'] = df3num['slash']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['slash_space']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['kr']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['ki']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['leie']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['zeros']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['4num']
df3num.loc[df3num['rent'].isnull(),'rent'] = df3num['kr_after']
dfr = df3num[pd.notnull(df3num['rent'])].reset_index(drop=True)
#df3num = df3num.style.set_properties(subset=['all'], **{'width': '300px'})

In [584]:
#dfr = dfr.style.set_properties(subset=['all'], **{'width': '300px'})
# extract rooms
# extract rooms
dfr['rooms'] = dfr['all'].str.split('var | v | rom | vär | vær | roms | vor | ettroms | V. | r. | vaer | sier | ver | aer | vier | sovevaer | sovevær | soveværelser ').str[0]
dfr.loc[~dfr['all'].str.contains('var | v | rom | vär | vær | roms | vor | ettroms | V. | r. | vaer | sier | ver | aer | vier | sovevaer | sovevær | soveværelser '), 'rooms'] = ''
dfr['rooms1'] = dfr['rooms'].str.split('(?<!\d)(\d{1})(?!\d)').str[1]
dfr['rooms1'] = dfr['rooms1'].replace(np.nan, '', regex=True)

dfr['second'] = dfr['rooms']
for i in range(0,len(dfr)):
    dfr.iloc[i, 14] = dfr.iloc[i, 14].replace(dfr.iloc[i,13],'')
dfr['rooms2'] = dfr['second'].str.split('(?<!\d)(\d{1})(?!\d)').str[1]

dfr['rooms'] = dfr['rooms2']
dfr.loc[dfr['rooms'].isnull(),'rooms'] = dfr['rooms1']
dfr = dfr.drop(['rooms1', 'rooms2', 'second'], axis=1)
# replace Hybelleil as one room
dfr.loc[dfr['all'].str.contains('Hybelleil'), 'rooms'] = '1'
dfr.loc[dfr['all'].str.contains('hyb'), 'rooms'] = '1'
# keep only number of rooms
dfr['rooms'] = dfr['rooms'].str.split('(?<!\d)(\d{1})(?!\d)').str[1]

#extract kitchen
dfr['kitchen'] = 0
dfr.loc[dfr['all'].str.contains('grovkjøkken | kjk | ki | kjokken | kj | Kjokken | kjok | k \ '), 'kitchen'] = 1
#extract hall
dfr['hall'] = 0
dfr.loc[dfr['all'].str.contains('hall'), 'hall'] = 1
#extract garage
dfr['garage'] = 0
dfr.loc[dfr['all'].str.contains('garasje'), 'garage'] = 1
#extract bathroom
dfr['bathroom'] = 0
dfr.loc[dfr['all'].str.contains('bad'), 'bathroom'] = 1
#extract balkony
dfr['balkony'] = 0
dfr.loc[dfr['all'].str.contains('balk'), 'balkony'] = 1

In [585]:
dfr['zip4'] = dfr['all'].str.extract(r'[/B][/m][/.](\d{4})(?!\d)', expand=False)
dfr['zip5'] = dfr['all'].str.extract(r'[/B][/m][^\S][[/.][^\S](\d{5})(?!\d)', expand=False)
dfr['zip6'] = dfr['all'].str.extract(r'[/B][/m][[/.](\d{5})(?!\d)', expand=False)
dfr['zip7'] = dfr['all'].str.extract(r'[/B][/m][^\S][[/.][^\S][/l][/e][/f][/t][^\S](\d{5})(?!\d)', expand=False)
dfr['zip'] = dfr['zip4']
dfr.loc[dfr['zip'].isnull(),'zip'] = dfr['zip5']
dfr.loc[dfr['zip'].isnull(),'zip'] = dfr['zip6']
dfr.loc[dfr['zip'].isnull(),'zip'] = dfr['zip7']
dfr['zip'] = dfr['zip'].replace(np.nan, '', regex=True)
dfr = dfr.drop(['zip4', 'zip5', 'zip6', 'zip7'], axis = 1) 

In [621]:
# create fin dataset
dfall = dfr[['date', 'all', 'rent', 'rooms', 'kitchen', 'hall', 'garage', 'bathroom', 'balkony', 'zip']].astype(str)
# encode norwegian
for i in range(0,len(dfall)):
    dfall.iloc[i,1] = unidecode(dfall.iloc[i,1])
# extract first Uppercase + 11 characters
dfall['area1'] = dfall['all'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area1'] = dfall['area1'].str.split(r'[^\S]').str[0]
dfall['area1'] = dfall['area1'].replace(np.nan, '', regex=True)
# extract second Uppercase + 11 characters
dfall['second'] = dfall['all']
for i in range(0,len(dfall)):
    dfall.iloc[i, 11] = dfall.iloc[i, 11].replace(dfall.iloc[i, 10],'')
dfall['area2'] = dfall['second'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area2'] = dfall['area2'].replace(np.nan, '', regex=True)
dfall.loc[dfall['area2'].str.contains('St'), 'area2'] = "Hansfjellet"
dfall['area2'] = dfall['area2'].str.split(r'[^\S]').str[0]
#extract third Uppercase + 11 characters
dfall['third'] = dfall['second']
for i in range(0,len(dfall)):
    dfall.iloc[i, 13] = dfall.iloc[i, 13].replace(dfall.iloc[i,12],'')
dfall['area3'] = dfall['third'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area3'] = dfall['area3'].replace(np.nan, '', regex=True)
dfall['area3'] = dfall['area3'].str.split(r'[^\S]').str[0]
#extract fourth Uppercase + 11 characters
dfall['fourth'] = dfall['third']
for i in range(0,len(dfall)):
    dfall.iloc[i, 15] = dfall.iloc[i, 15].replace(dfall.iloc[i,14],'')
dfall['area4'] = dfall['fourth'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area4'] = dfall['area4'].replace(np.nan, '', regex=True)
dfall['area4'] = dfall['area4'].str.split(r'[^\S]').str[0]
#extract fifth Uppercase + 11 characters
dfall['fifth'] = dfall['fourth']
for i in range(0,len(dfall)):
    dfall.iloc[i, 17] = dfall.iloc[i, 17].replace(dfall.iloc[i,16],'')
dfall['area5'] = dfall['fifth'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area5'] = dfall['area5'].replace(np.nan, '', regex=True)
dfall['area5'] = dfall['area5'].str.split(r'[^\S]').str[0]
#extract sixth Uppercase + 11 characters
dfall['sixth'] = dfall['fifth']
for i in range(0,len(dfall)):
    dfall.iloc[i, 19] = dfall.iloc[i, 19].replace(dfall.iloc[i,18],'')
dfall['area6'] = dfall['sixth'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area6'] = dfall['area6'].replace(np.nan, '', regex=True)
dfall['area6'] = dfall['area6'].str.split(r'[^\S]').str[0]
#extract seventh Uppercase + 11 characters
dfall['seventh'] = dfall['sixth']
for i in range(0,len(dfall)):
    dfall.iloc[i, 21] = dfall.iloc[i, 21].replace(dfall.iloc[i,20],'')
dfall['area7'] = dfall['seventh'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area7'] = dfall['area7'].replace(np.nan, '', regex=True)
dfall['area7'] = dfall['area7'].str.split(r'[^\S]').str[0]
#extract eightth Uppercase + 11 characters
dfall['eightth'] = dfall['seventh']
for i in range(0,len(dfall)):
    dfall.iloc[i, 23] = dfall.iloc[i, 23].replace(dfall.iloc[i,22],'')
dfall['area8'] = dfall['eightth'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area8'] = dfall['area8'].replace(np.nan, '', regex=True)
dfall['area8'] = dfall['area8'].str.split(r'[^\S]').str[0]
#extract ninth Uppercase + 11 characters
dfall['ninth'] = dfall['eightth']
for i in range(0,len(dfall)):
    dfall.iloc[i, 25] = dfall.iloc[i, 25].replace(dfall.iloc[i,24],'')
dfall['area9'] = dfall['ninth'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area9'] = dfall['area9'].replace(np.nan, '', regex=True)
dfall['area9'] = dfall['area9'].str.split(r'[^\S]').str[0]
#extract tenth Uppercase + 11 characters
dfall['tenth'] = dfall['ninth']
for i in range(0,len(dfall)):
    dfall.iloc[i, 27] = dfall.iloc[i, 27].replace(dfall.iloc[i,26],'')
dfall['area10'] = dfall['tenth'].str.extract(pat = '([A-Z][a-z]..........)')
dfall['area10'] = dfall['area10'].replace(np.nan, '', regex=True)
dfall['area10'] = dfall['area10'].str.split(r'[^\S]').str[0]
dfall = dfall.drop(['second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eightth', 'ninth', 'tenth'], axis = 1) 
#replace all area names that are certainly not areas
areas = ['area1', 'area2', 'area3', 'area4', 'area5', 'area6', 'area7', 'area8', 'area9', 'area10']
for i in range(0,10):
    dfall = dfall.replace({areas[i] : { 'Hybelleil' : '', 'Leie' : '', 'Innsk' : '', 'Bm' : '', 'Juli' : '', 
                                       'Obligasjon' : '', 'Innsk./leie' : '', 'Kjokken' : '', 'Lele' : '', 
                                       'Bis' : '', 'Mtt' : '', 'Pris' : '', 'Ca.' : '', 'Mod' : '', 'Bm.5479' : '', 
                                       'Minst' : '', 'Kr' : '', 'Inn' : '', 'Innkl' : '', 'Tit' : '', 'Tlf' : '', 
                                       'Stor' : '', 'Flott' : '', 'Leilighet' : '', 'Skr' : '', 'Leif' : '', 'Chr' : '',
                                      'Voss' : '', 'Castberg' : '', 'Mob' : ''}})

In [622]:
#extract living rooms
dfall['living_rooms'] = dfall['all'].str.split('stuer').str[0]
dfall.loc[~dfall['all'].str.contains('stuer'), 'living_rooms'] = ''
dfall['living_rooms'] = dfall['living_rooms'].str.split('(?<!\d)(\d{1})(?!\d)').str[1]
dfall['living_rooms'] = dfall['living_rooms'].replace(np.nan, '0', regex=True)

In [623]:
#dfall.style.set_properties(subset=['all'], **{'width': '300px'})
dfall = dfall.sort_values('date').reset_index(drop=True)
dfall = dfall.drop(['all'], axis=1)
dfall

,date,rent,rooms,kitchen,hall,garage,bathroom,balkony,zip,area1,area2,area3,area4,area5,area6,area7,area8,area9,area10,living_rooms
0,tl60.06.01.1.png,350,4,0,0,0,0,1,16224,,,,,,,,,,,0
1,tl60.06.01.1.png,300,3,0,0,0,0,0,16393,Sinsen,Ekeberg,,,,Haster,,Borett,,,0
2,tl60.06.01.2.png,1000,3,0,0,0,0,0,,,,,,,,,,,,0
3,tl60.06.03.1.png,300,3,1,0,0,1,0,19613,Dann,Blindern,Gode,,,,,,,,0
4,tl60.06.03.2.png,340,3,1,0,0,1,0,5479,,Roa,,,,,,,,,0
5,tl60.06.08.2.png,200,1,0,0,0,0,0,,Sentralt,,,,,,,,,,0
6,tl60.06.08.2.png,350,3,0,0,0,0,0,,,Arvoll,,,,,Hartsang,Tollbugt,,,0
7,tl60.06.10.1.png,750,3,0,0,0,0,1,,Frognerstran,Meget,Iste,Bolrik,,Bift,Kun,,,,0
8,tl60.06.10.1.png,260,2,0,0,0,0,0,,Aksjeleiligh,Fjellhus,Prost,Hallings,Aker,Noen,,,Naerm,,0
9,tl60.06.10.1.png,1000,4,0,0,0,1,0,,,Blommenholm,Henv,,,,Kirkegt,,,,3


In [624]:
dfall.to_excel('tl_01-10.06.1960.xlsx')